In [ ]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
import copy

np.set_printoptions(precision=3, suppress=True)

# Step 1 – Environment setup
env = gym.make('Taxi-v3', render_mode='ansi')
obs, info = env.reset()

print("Initial State:", obs)
print("Observation space size:", env.observation_space.n)  # 500 states
print("Action space size:", env.action_space.n)            # 6 actions

# Original transition model (keep a copy)
P_orig = env.unwrapped.P
reward_min = min({r for s in P_orig for a in P_orig[s] for (_, _, r, _) in P_orig[s][a]})
reward_max = max({r for s in P_orig for a in P_orig[s] for (_, _, r, _) in P_orig[s][a]})
print("Reward range:", (reward_min, reward_max))

print(env.render())

# -------- helper: make modified reward model --------
def modify_rewards(P_base, time_penalty=-1, illegal_penalty=-10, success_reward=20):
    """
    Returns a deep-copied transition model with modified rewards.

    Taxi-v3 default:
      - step:      -1
      - illegal:  -10
      - success:  +20
    We map these to new values.
    """
    P_new = copy.deepcopy(P_base)
    for s in P_new:
        for a in P_new[s]:
            new_list = []
            for prob, next_state, reward, done in P_new[s][a]:
                if reward == -1:
                    r_new = time_penalty
                elif reward == -10:
                    r_new = illegal_penalty
                elif reward == 20:
                    r_new = success_reward
                else:
                    r_new = reward
                new_list.append((prob, next_state, r_new, done))
            P_new[s][a] = new_list
    return P_new

# -------- Step 3 – Value Iteration (now takes P as argument) --------
def value_iteration(env, P, discount_factor=0.99, theta=1e-6, max_iterations=10000):
    nS = env.observation_space.n
    nA = env.action_space.n
    V = np.zeros(nS)

    deltas = []  # record max change each iteration

    for i in range(max_iterations):
        delta = 0.0
        for s in range(nS):
            q_sa = np.zeros(nA)
            for a in range(nA):
                for prob, next_state, reward, done in P[s][a]:
                    q_sa[a] += prob * (reward + discount_factor * V[next_state])
            new_v = np.max(q_sa)
            delta = max(delta, abs(new_v - V[s]))
            V[s] = new_v
        deltas.append(delta)
        if delta < theta:
            break

    policy = extract_policy_from_v(env, P, V, discount_factor)
    return V, policy, i + 1, deltas

# -------- Step 4 – Extract policy from V (also uses P) --------
def extract_policy_from_v(env, P, V, discount_factor=0.99):
    nS = env.observation_space.n
    nA = env.action_space.n
    policy = np.zeros((nS, nA))

    for s in range(nS):
        q_sa = np.zeros(nA)
        for a in range(nA):
            for prob, next_state, reward, done in P[s][a]:
                q_sa[a] += prob * (reward + discount_factor * V[next_state])
        best_action = np.argmax(q_sa)
        policy[s] = np.eye(nA)[best_action]
    return policy

# -------- Evaluate policy: success rate + avg steps (using real env) --------
def evaluate_policy(env, policy, n_episodes=500):
    success = 0
    total_steps_success = 0

    for _ in range(n_episodes):
        obs, _ = env.reset()
        done = False
        steps = 0

        while not done:
            action = np.argmax(policy[obs])
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            steps += 1

            # success = reached passenger drop-off successfully (reward > 0 at end)
            if done and reward > 0:
                success += 1
                total_steps_success += steps

    success_rate = success / n_episodes
    avg_steps_success = (total_steps_success / success) if success > 0 else float('inf')
    return success_rate, avg_steps_success

# -------- Step 5 – Plot helpers --------
def plot_values(env, V, label=""):
    plt.figure(figsize=(10, 4))
    plt.plot(V)
    title = "Value Function for Taxi-v3"
    if label:
        title += f" ({label})"
    plt.title(title)
    plt.xlabel("State (0–499)")
    plt.ylabel("Value")
    plt.grid(True)
    plt.show()

def plot_policy(env, policy, label=""):
    nS = env.observation_space.n
    actions = np.argmax(policy, axis=1)
    plt.figure(figsize=(10, 4))
    plt.bar(np.arange(nS), actions)
    title = "Greedy Policy (Best Action per State)"
    if label:
        title += f" ({label})"
    plt.title(title)
    plt.xlabel("State Index")
    plt.ylabel("Action (0–5)")
    plt.show()

# -------- Step 6 – Test different reward structures --------
if __name__ == "__main__":
    gamma = 0.99  # keep gamma fixed; change only reward structure

    # (name, time_penalty, illegal_penalty, success_reward)
    reward_settings = [
        ("default",        -1,  -10,  20),
        ("low_time_cost",  -0.2, -10, 20),
        ("harsh_illegal",  -1,  -30, 20),
        ("high_success",   -1,  -10, 40),
    ]

    all_V = {}
    all_iters = {}
    success_rates = {}
    avg_steps = {}
    all_deltas = {}

    for name, t_pen, ill_pen, succ_rew in reward_settings:
        print(f"\n=== Running Value Iteration with reward setting: {name} ===")
        P_mod = modify_rewards(P_orig,
                               time_penalty=t_pen,
                               illegal_penalty=ill_pen,
                               success_reward=succ_rew)

        V_opt, policy_opt, iters, deltas = value_iteration(
            env, P_mod, discount_factor=gamma
        )

        all_V[name] = V_opt
        all_iters[name] = iters
        all_deltas[name] = deltas

        print(f"Converged in {iters} iterations for setting '{name}'")

        rate, avg_steps_succ = evaluate_policy(env, policy_opt, n_episodes=500)
        success_rates[name] = rate
        avg_steps[name] = avg_steps_succ

        print(f"Success rate ({name}): {rate*100:.2f}%")
        print(f"Avg steps / successful episode ({name}): {avg_steps_succ:.2f}")

        plot_values(env, V_opt, label=name)
        plot_policy(env, policy_opt, label=name)

    # --- Comparison plots ---

    # Value iteration convergence (delta)
    plt.figure(figsize=(10, 4))
    for name in reward_settings:
        label = name[0]  # first element of tuple is the name
        plt.plot(all_deltas[label], label=label)
    plt.title("Value Iteration Convergence for Different Reward Structures")
    plt.xlabel("Iteration")
    plt.ylabel("Delta (max |V_new - V_old|)")
    plt.yscale("log")
    plt.grid(True)
    plt.legend()
    plt.show()

    # Success rate vs reward structure
    plt.figure(figsize=(6, 4))
    labels = [cfg[0] for cfg in reward_settings]
    sr_vals = [success_rates[name] * 100 for name in labels]
    plt.bar(labels, sr_vals)
    plt.title("Success Rate vs Reward Structure")
    plt.ylabel("Success Rate (%)")
    plt.xticks(rotation=20)
    plt.show()

    # Avg steps vs reward structure
    plt.figure(figsize=(6, 4))
    steps_vals = [avg_steps[name] for name in labels]
    plt.bar(labels, steps_vals)
    plt.title("Average Steps per Successful Episode vs Reward Structure")
    plt.ylabel("Avg Steps (successful episodes)")
    plt.xticks(rotation=20)
    plt.show()

    # Iterations to converge vs reward structure
    plt.figure(figsize=(6, 4))
    it_vals = [all_iters[name] for name in labels]
    plt.bar(labels, it_vals)
    plt.title("Convergence Speed vs Reward Structure")
    plt.ylabel("Iterations to Converge")
    plt.xticks(rotation=20)
    plt.show()
